In [2]:
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

In [4]:
chunksize = 1000
test = None
for count, chunk in enumerate(pd.read_csv("/home/owner/Documents/DEV/graphs/graphs_and_structural_analysis/Santander_Product_Recommendation/test_ver2.csv",
                         chunksize=chunksize)):
    # process each chunk here
    if (count + 1) % 100 == 0: print(f"step {count + 1} by 1000")
    if test is not None: 
        test = pd.concat([test, chunk])
    else: test = chunk

FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:
col_names = {"ncodpers":"cust_id", "ind_empleado":"emp_index","pais_residencia":"residence",
            "sexo":"sex","fecha_alta":"first_date","ind_nuevo":"new_cust","antiguedad":"seniority",
            "indrel":"is_primary","ult_fec_cli_1t":"last_primary_date","indrel_1mes":"cust_type",
            "tiprel_1mes":"cust_rel_type","indresi":"residence_index","indext":"foreigner_index",
            "conyuemp":"spouse_index","canal_entrada":"channel","cod_prov":"province","nomprov":"province_name",
            "ind_actividad_cliente":"active_index","renta":"income","segmento":"segment"}

test.rename(col_names, axis = 1, inplace = True)

In [ ]:
test.age = pd.to_numeric(test.age, errors='coerce')
test.income = pd.to_numeric(test.income, errors='coerce')
test.seniority = pd.to_numeric(test.seniority, errors='coerce')
test.first_date = pd.to_datetime(test.first_date, errors = 'coerce')
test.cust_type = pd.to_numeric(test.cust_type, errors='coerce')
test['fecha_dato'] = pd.to_datetime(test['fecha_dato'])

In [ ]:
test.isna().sum() / len(test)

fecha_dato           0.000000
cust_id              0.000000
emp_index            0.002032
residence            0.002032
sex                  0.002037
age                  0.002032
first_date           0.002032
new_cust             0.002032
seniority            0.002032
is_primary           0.002032
last_primary_date    0.998183
cust_type            0.010975
cust_rel_type        0.010975
residence_index      0.002032
foreigner_index      0.002032
spouse_index         0.999868
channel              0.013638
indfall              0.002032
tipodom              0.002032
province             0.006858
province_name        0.006858
active_index         0.002032
income               0.204756
segment              0.013876
ind_ahor_fin_ult1    0.000000
ind_aval_fin_ult1    0.000000
ind_cco_fin_ult1     0.000000
ind_cder_fin_ult1    0.000000
ind_cno_fin_ult1     0.000000
ind_ctju_fin_ult1    0.000000
ind_ctma_fin_ult1    0.000000
ind_ctop_fin_ult1    0.000000
ind_ctpp_fin_ult1    0.000000
ind_deco_f

In [ ]:
# For features with missing values percentage < 10%, use the most common factor level to impute
cols = ['emp_index','residence','sex','first_date','new_cust','is_primary',"cust_type","cust_rel_type",
       "province","province_name","active_index","channel","segment"]

for i in cols:
    test.loc[test[i].isnull(), i] = test[i].value_counts().index[0]
    
# For features with missing value accounts for over 10%, impute their missing values based on the mean at different dimensions
# For Income, impute the missings using the medians of Customer Segment and Province
test['income'].fillna(test['income'].mean(), inplace = True)


/tmp/ipykernel_12030/386424114.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['income'].fillna(df['income'].mean(), inplace = True)


In [ ]:
# Drop the last primary date and spouse index fields given over 99% missing values
test.drop(['last_primary_date','spouse_index'], axis = 1, inplace = True)

In [ ]:
test.head()

,fecha_dato,cust_id,emp_index,residence,sex,age,first_date,new_cust,seniority,is_primary,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35.0,2015-01-12,0.0,6.0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23.0,2012-08-10,0.0,35.0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23.0,2012-08-10,0.0,35.0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22.0,2012-08-10,0.0,35.0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23.0,2012-08-10,0.0,35.0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


# Feature engineering

In [ ]:
# 1) Age group: group the age for lifestages: young, adult, senior, elder
age_group = [0, 20, 45, 65, 100]
age_labels = ['young', 'adult', 'senior', 'elder']

test['age_grouped'] = pd.cut(test.age, bins = age_group, labels = age_labels)

In [ ]:
# 2) Income Group: group the incomes at Low, Ordinary, Median-high and High classes at (real case may be more complex)
import numpy as np
df_income_group = [0, np.nanpercentile(test['income'], 25), np.nanpercentile(test['income'], 50), np.nanpercentile(test['income'], 75), np.nanpercentile(test['income'], 100)]
income_labels = ['Low','Ordinary','Median-high','High']

test['income_grouped'] = pd.cut(test['income'], df_income_group, labels = income_labels)

In [ ]:
categorical = test.columns[(test.dtypes == "object") | (test.dtypes == "category")]

In [ ]:
categorical

Index(['emp_index', 'residence', 'sex', 'cust_rel_type', 'residence_index',
       'foreigner_index', 'channel', 'indfall', 'province_name', 'segment',
       'age_grouped', 'income_grouped'],
      dtype='object')

In [ ]:
for column in categorical:
    test = pd.concat([test, pd.get_dummies(test[column])], axis=1)
    test.drop(columns=column, inplace=True)

: 

In [ ]:
test.head()

,fecha_dato,cust_id,emp_index,residence,sex,age,first_date,new_cust,seniority,is_primary,...,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,age_grouped,income_grouped
0,2015-01-28,1375586,N,ES,H,35.0,2015-01-12,0.0,6.0,1.0,...,0,0,0,0,0,0.0,0.0,0,adult,Ordinary
1,2015-01-28,1050611,N,ES,V,23.0,2012-08-10,0.0,35.0,1.0,...,0,0,0,0,0,0.0,0.0,0,adult,Low
2,2015-01-28,1050612,N,ES,V,23.0,2012-08-10,0.0,35.0,1.0,...,0,0,0,0,0,0.0,0.0,0,adult,Ordinary
3,2015-01-28,1050613,N,ES,H,22.0,2012-08-10,0.0,35.0,1.0,...,0,0,0,0,0,0.0,0.0,0,adult,Ordinary
4,2015-01-28,1050614,N,ES,V,23.0,2012-08-10,0.0,35.0,1.0,...,0,0,0,0,0,0.0,0.0,0,adult,Median-high


# А теперь положим в нормальный файл

In [ ]:
%pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 14.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
test.to_feather("train.feather")

ArrowInvalid: ("Could not convert '1.0' with type str: tried to convert to double", 'Conversion failed for column cust_type with type object')